In [1]:
from gpumon.nvidia_dmon import nvidia_run_dmon_poll, create_influxdb_writer
from influxdb import InfluxDBClient, DataFrameClient
import pandas
from itertools import filterfalse
from toolz import merge,curry, compose, pipe

ImportError: cannot import name 'create_influxdb_writer'

In [28]:
nvidia_run_dmon_poll(print)

b'#Date       Time        gpu   pwr  temp    sm   mem   enc   dec  mclk  pclk\n'
b'#YYYYMMDD   HH:MM:SS    Idx     W     C     %     %     %     %   MHz   MHz\n'
b' 20171019   11:05:01      0    14    30     0     0     0     0   324   405\n'
b' 20171019   11:05:01      1    13    27     0     0     0     0   324   405\n'
b' 20171019   11:05:01      2    14    32     0     0     0     0   324   405\n'
b' 20171019   11:05:01      3    14    34     0     0     0     0   324   405\n'
b' 20171019   11:05:03      0    14    30     0     0     0     0   324   405\n'
b' 20171019   11:05:03      1    13    27     0     0     0     0   324   405\n'
b' 20171019   11:05:03      2    14    32     0     0     0     0   324   405\n'
b' 20171019   11:05:03      3    14    34     0     0     0     0   324   405\n'
b' 20171019   11:05:04      0    14    30     0     0     0     0   324   405\n'
b' 20171019   11:05:04      1    13    27     0     0     0     0   324   405\n'
b' 20171019   11:05:04      

KeyboardInterrupt: 

In [2]:
client = InfluxDBClient('localhost', 8086, 'admin', 'admin', 'experiment1')

In [3]:
client.create_database('experiment1')

In [4]:
def to_json(gpu_prop_list):
    print(gpu_prop_list)
    return { "time": gpu_prop_list[0].strftime('%Y-%m-%dT%H:%M:%S%z'),
            "tags": {
                "gpu": gpu_prop_list[1],
            },
             "fields": {
                "pwr": gpu_prop_list[2],
                "temp": gpu_prop_list[3],
                "sm": gpu_prop_list[4],
                "mem": gpu_prop_list[5],
                "enc": gpu_prop_list[6],
                "dec": gpu_prop_list[7],
                "mclk": gpu_prop_list[8],
                "pclk": gpu_prop_list[9],
             }
           }

In [5]:
def bytes_to_string(bytes_list):
    return "".join(map(chr, bytes_list))

In [6]:
def create_influxdb_writer(influxdb_client, measurement):
    mes_dict = {"measurement": measurement}
    def to_influxdf(*data_dicts):
        print(merge(mes_dict, *data_dicts))
        res = influxdb_client.write_points([merge(mes_dict, *data_dicts)])
        print(res)
    return to_influxdf

In [12]:
@curry
def call_when(write_func, predicate_func, line):
    if predicate_func(line):
        write_func(line)

In [13]:
to_influxdf = create_influxdb_writer(client, "gpu_load")

In [14]:
write_to_db = compose(to_influxdf, 
                      to_json, 
                      parse_line)

In [15]:
to_db = compose(call_when(write_to_db, lambda x: x is not None and '#' not in x), 
                bytes_to_string)

In [16]:
nvidia_run_dmon_poll(to_db)

[datetime.datetime(2017, 10, 19, 16, 41, 10), 0, 14, 33, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:10', 'tags': {'gpu': 0}, 'fields': {'pwr': 14, 'temp': 33, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 10), 1, 14, 30, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:10', 'tags': {'gpu': 1}, 'fields': {'pwr': 14, 'temp': 30, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 10), 2, 14, 35, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:10', 'tags': {'gpu': 2}, 'fields': {'pwr': 14, 'temp': 35, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 10), 3, 14, 38, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:10', 'tags': {'gpu': 3}, 'fields': {'pwr': 14, 'temp': 38, 'sm': 0, 'mem': 0, 

[datetime.datetime(2017, 10, 19, 16, 41, 20), 0, 14, 33, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:20', 'tags': {'gpu': 0}, 'fields': {'pwr': 14, 'temp': 33, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 20), 1, 13, 30, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:20', 'tags': {'gpu': 1}, 'fields': {'pwr': 13, 'temp': 30, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 20), 2, 14, 35, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:20', 'tags': {'gpu': 2}, 'fields': {'pwr': 14, 'temp': 35, 'sm': 0, 'mem': 0, 'enc': 0, 'dec': 0, 'mclk': 324, 'pclk': 405}}
True
[datetime.datetime(2017, 10, 19, 16, 41, 20), 3, 14, 37, 0, 0, 0, 0, 324, 405]
{'measurement': 'gpu_load', 'time': '2017-10-19T16:41:20', 'tags': {'gpu': 3}, 'fields': {'pwr': 14, 'temp': 37, 'sm': 0, 'mem': 0, 

KeyboardInterrupt: 

In [17]:
client.query('select * from gpu_load;')

ResultSet({'('gpu_load', None)': [{'time': '2017-10-19T16:41:10Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 14, 'sm': 0, 'temp': 33}, {'time': '2017-10-19T16:41:10Z', 'dec': 0, 'enc': 0, 'gpu': '3', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 14, 'sm': 0, 'temp': 38}, {'time': '2017-10-19T16:41:10Z', 'dec': 0, 'enc': 0, 'gpu': '2', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 14, 'sm': 0, 'temp': 35}, {'time': '2017-10-19T16:41:10Z', 'dec': 0, 'enc': 0, 'gpu': '1', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 14, 'sm': 0, 'temp': 30}, {'time': '2017-10-19T16:41:11Z', 'dec': 0, 'enc': 0, 'gpu': '2', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 14, 'sm': 0, 'temp': 35}, {'time': '2017-10-19T16:41:11Z', 'dec': 0, 'enc': 0, 'gpu': '1', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 13, 'sm': 0, 'temp': 30}, {'time': '2017-10-19T16:41:11Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'mclk': 324, 'mem': 0, 'pclk': 405, 'pwr': 13, 'sm': 0, 'temp': 33}, {'time': '2017-10-19T16:41:1

In [18]:
df_client = DataFrameClient('localhost', 8086, 'admin', 'admin', 'experiment1')

In [19]:
df_client.query("select * from gpu_load;")['gpu_load']

,dec,enc,gpu,mclk,mem,pclk,pwr,sm,temp
2017-10-19 16:41:10+00:00,0,0,0,324,0,405,14,0,33
2017-10-19 16:41:10+00:00,0,0,3,324,0,405,14,0,38
2017-10-19 16:41:10+00:00,0,0,2,324,0,405,14,0,35
2017-10-19 16:41:10+00:00,0,0,1,324,0,405,14,0,30
2017-10-19 16:41:11+00:00,0,0,2,324,0,405,14,0,35
2017-10-19 16:41:11+00:00,0,0,1,324,0,405,13,0,30
2017-10-19 16:41:11+00:00,0,0,0,324,0,405,13,0,33
2017-10-19 16:41:11+00:00,0,0,3,324,0,405,14,0,37
2017-10-19 16:41:12+00:00,0,0,0,324,0,405,14,0,33
2017-10-19 16:41:12+00:00,0,0,3,324,0,405,15,0,37


In [20]:
client.get_list_database()

[{'name': '_internal'},
 {'name': 'mydb'},
 {'name': 'example'},
 {'name': 'experiment1'}]

In [63]:
client.drop_database('experiment1')

In [64]:
client.get_list_database()

[{'name': '_internal'}, {'name': 'mydb'}, {'name': 'example'}]